In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pymongo
client = pymongo.MongoClient()
db = client.metrics

In [3]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging
import datetime

In [4]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [5]:
#with open(training_pickled, "rb+") as f:
#    tagged_essays = pickle.load(f)
#len(tagged_essays)

## Load Training Data - Essays Tagged with Codes By a Bi-Directional RNN

In [6]:
import dill
fname = predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(fname, "rb") as f:
    pred_tagged_essays = dill.load(f)

#len(tagged_essays), 
len(pred_tagged_essays) # should be 902

902

In [7]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-07-29 13:14:44.078527


In [8]:
CAUSER = "Causer"
RESULT = "Result"
EXPLICIT = "explicit"
CAUSER_EXPLICIT = "Causer_Explicit"
EXPLICIT_RESULT = "Explicit_Result"
CAUSER_EXPLICIT_RESULT = "Causer_Explicit_Result"
CAUSER_RESULT = "Causer_Result"

In [9]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in pred_tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
#TODO - don't ignore Anaphor, other and rhetoricals here
cr_tags = list((t for t in tag_freq.keys() if ( "->" in t) and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))

vtags = set(regular_tags)
#vtags.add(EMPTY_TAG)
#cr_tags = vtags

len(unique_words)

1641

In [10]:
regular_tags

['50', '5', '4', '11', '13', '3', '7', '1', '6', '14', '2', '12', '5b']

In [11]:
cr_tags

['Causer:5->Result:50',
 'Causer:7->Result:50',
 'Causer:3->Result:4',
 'Causer:1->Result:50',
 'Causer:13->Result:50',
 'Causer:11->Result:50',
 'Causer:6->Result:50',
 'Causer:3->Result:5',
 'Causer:4->Result:14',
 'Causer:3->Result:1',
 'Causer:1->Result:3',
 'Causer:3->Result:50',
 'Causer:4->Result:5',
 'Causer:3->Result:7',
 'Causer:6->Result:7',
 'Causer:11->Result:12',
 'Causer:12->Result:13',
 'Causer:7->Result:5b',
 'Causer:5b->Result:50',
 'Causer:50->Result:50',
 'Causer:1->Result:2',
 'Causer:2->Result:3',
 'Causer:11->Result:3',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:5->Result:5b',
 'Causer:1->Result:7',
 'Causer:7->Result:14',
 'Causer:5->Result:7',
 'Causer:11->Result:6',
 'Causer:13->Result:14',
 'Causer:3->Result:13',
 'Causer:3->Result:14',
 'Causer:4->Result:50',
 'Causer:14->Result:50',
 'Causer:1->Result:14',
 'Causer:4->Result:5b',
 'Causer:1->Result:4',
 'Causer:6->Result:14',
 'Causer:5b->Result:7',
 'Causer:3->Result:5b',
 'Causer:13->Resul

In [12]:
[k for k in sorted(tag_freq.keys()) if "->" not in k]

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Anaphor',
 'COMPILED',
 'Causer',
 'Causer:1',
 'Causer:11',
 'Causer:12',
 'Causer:13',
 'Causer:14',
 'Causer:2',
 'Causer:3',
 'Causer:4',
 'Causer:5',
 'Causer:50',
 'Causer:5b',
 'Causer:6',
 'Causer:7',
 'Causer:Anaphor',
 'Causer:other',
 'Causer:rhetorical',
 'Result',
 'Result:1',
 'Result:11',
 'Result:12',
 'Result:13',
 'Result:14',
 'Result:2',
 'Result:3',
 'Result:4',
 'Result:5',
 'Result:50',
 'Result:5b',
 'Result:6',
 'Result:7',
 'Result:Anaphor',
 'Result:other',
 'Result:rhetorical',
 'explicit',
 'other',
 'rhetorical']

In [13]:
from Decorators import memoize
from nltk import PorterStemmer
from NgramGenerator import compute_ngrams

In [2]:
def assign_predicted_tags_to_essay_feats(essay_feats, pred_tagged_essays):
    essay2pred = {}
    for essay in pred_tagged_essays:
        essay2pred[essay.name] = essay

    for essay in essay_feats:
        pred_essay = essay2pred[essay.name]
        for six, sent in enumerate(essay.sentences):
            pred_sent = pred_essay.pred_tagged_sentences[six]
            assert len(pred_sent) == len(sent), "Miss-match on sentences"
            for wix, wd in enumerate(sent):
                wd.predicted_tag = pred_sent[wix]
    
#assign_predicted_tags_to_essay_feats(essay_feats, pred_tagged_essays)

## Parsing Logic

In [14]:
class Stack(object):
    def __init__(self, verbose=False):    
        self.stack = []
        self.verbose = verbose
    
    def tos(self):
        if self.len() == 0:
            return None
        #assert self.len() > 0, "Can't peek when stack is empty"
        return self.stack[-1]
    
    def pop(self):
        assert self.len() > 0, "Can't pop when stack is empty"
        item = self.stack.pop()
        if self.verbose:
            print("POPPING: %s" % item)
            print("LEN:     %i" % len(self.stack))
        return item
    
    def push(self, item):
        self.stack.append(item)
        if self.verbose:
            print("PUSHING: %s" % item)
            print("LEN:     %i" % len(self.stack))
    
    def len(self):
        return len(self.stack)

    def contains(self, item):
        return item in self.stack
    
    def __repr__(self):
        return "|".join(self.stack)
    
    def clone(self):
        clone = Stack(self.verbose)
        clone.stack = list(self.stack)
        return clone

In [15]:
ROOT = "root"

def norm_arc(arc):
    return tuple(sorted(arc))

def norm_arcs(arcs):
    return set(map(norm_arc, arcs))

class Parser(object):
    def __init__(self, stack):
        self.stack = stack
        self.arcs = []
        self.normed_arcs = set()
        # nodes with heads
        self.children = set()
        self.actions = []
        
    def get_dependencies(self):
        return [(l,r) for (l,r) in self.arcs if r != ROOT and l != ROOT]
        
    def left_arc(self, buffer):
        tos = self.stack.pop()
        #Pre-condition
        #assert self.has_head(tos) == False
        arc = (tos,buffer)
        n_arc = norm_arc(arc)
        assert n_arc not in self.normed_arcs, "Arc already processed %s" % (n_arc)
        self.arcs.append(arc)
        self.normed_arcs.add(arc)
        self.children.add(tos)
        self.actions.append("L ARC   : " + tos + "->" + buffer)
        
    def right_arc(self, buffer):
        tos = self.stack.tos()
        #normalize arc
        arc = (buffer,tos)
        n_arc = norm_arc(arc)
        assert n_arc not in self.normed_arcs, "Arc already processed %s" % (n_arc)
        self.arcs.append(arc)
        self.normed_arcs.add(n_arc)
        self.actions.append("R ARC   : " + tos + "<-" + buffer)
        self.children.add(buffer)
        self.stack.push(buffer)
        
    def reduce(self):
        tos = self.stack.pop()
        #assert self.has_head(tos) == True
        self.actions.append("REDUCE  : Pop  %s" % tos)
        
    def shift(self, buffer):
        self.stack.push(buffer)
        self.actions.append("SHIFT   : Push %s" % buffer)
    
    def skip(self, buffer):
        self.actions.append("SKIP    : item %s" % buffer)
    
    def has_head(self, item):
        return item in self.children
    
    def in_stack(self, item):
        return self.stack.contains(item)
    
    def clone(self):
        clone = Parser(self.stack.clone())
        clone.arcs = list(self.arcs)
        clone.normed_arcs = set(self.normed_arcs)
        # nodes with heads
        clone.children = set(self.children)
        clone.actions = list(self.actions)
        return clone

In [16]:
from collections import defaultdict

SHIFT = "Shift"
REDUCE = "Reduce"
LARC = "LArc"
RARC = "Rarc"
SKIP = "Skip"

class Oracle(object):
    
    def __init__(self, crels, parser):
        self.parser = parser
        self.raw_crels = crels
        self.crels = norm_arcs(crels) # type: Set[Tuple[str,str]]
        self.mapping = self.build_mappings(crels)
    
    def build_mappings(self, pairs):
        mapping = defaultdict(set)
        for c,res in pairs:
            mapping[c].add(res)
            mapping[res].add(c)
        return mapping

    def should_continue(self, action):
        # continue parsing if REDUCE or LARC
        return action in (REDUCE,LARC)
    
    def remove_relation(self, a,b):
        self.mapping[a].remove(b)
        if len(self.mapping[a]) == 0:
            del self.mapping[a]
        self.mapping[b].remove(a)
        if len(self.mapping[b]) == 0:
            del self.mapping[b]
    
    def consult(self, tos, buffer):
        """
        Performs optimal decision for parser
        If true, continue processing, else Consume Buffer
        """
        parser = self.parser
        a,b = norm_arc((tos, buffer))
        if (a,b) in self.crels:
            # TOS has arcs remaining? If so, we need RARC, else LARC
            if len(self.mapping[tos]) == 1:
                return LARC
            else:
                return RARC
        else:
            if buffer not in self.mapping:
                return SKIP
            # If the buffer has relations further down in the stack, we need to POP the TOS
            for item in self.mapping[buffer]:
                if item == tos:
                    continue
                if parser.in_stack(item):
                    return REDUCE
            #end for
            #ELSE
            return SHIFT
        
    def execute(self, action, tos, buffer):
        """
        Performs optimal decision for parser
        If true, continue processing, else Consume Buffer
        """
        parser = self.parser
        if action == LARC:
            parser.left_arc(buffer)
            self.remove_relation(tos, buffer)
        elif action == RARC:
            parser.right_arc(buffer)
            self.remove_relation(tos, buffer)
        elif action == REDUCE:
            parser.reduce()
        elif action == SHIFT:
            parser.shift(buffer)
        elif action == SKIP:
            pass
        else:
            raise Exception("Unknown parsing action %s" % action)
        return self.should_continue(action)
    
    def tos(self):
        return self.parser.stack.tos()
    
    def is_stack_empty(self):
        return self.parser.stack.len() == 0
    
    def clone(self):
        clone = Oracle(set(self.raw_crels), self.parser.clone())
        clone.mapping = dict(self.mapping.items())
        return clone

In [17]:
class FeatureExtractor(object):
    def __init__(self, extractors):
        self.extractors = extractors
    
    def extract(self, predicted_tag, word_seq, positive_val=1):
        """
        word: str
        word_seq: List[Word]
        
        returns: List[Dict{str,float}]
        """
        fts = dict()
        for ext in self.extractors:
            new_feats = ext(predicted_tag, word_seq, positive_val)
            fts.update(new_feats)
        return fts
    
def bag_of_word_extractor(predicted_tag, word_seq, positive_val):
    feats = {}
    for word in word_seq:
        feats["bow_" + word] = positive_val
    return feats

def bag_of_word_plus_tag_extractor(predicted_tag, word_seq, positive_val):
    feats = {}
    for word in word_seq:
        feats["bow_" + word + "_tag_" + predicted_tag] = positive_val
    return feats

feat_extractor = FeatureExtractor([
    bag_of_word_extractor,
    bag_of_word_plus_tag_extractor,
])

In [18]:
#def normalize(code):
#    if not code.endswith("b"):
#        code += " "
#   return code.rjust(3, "0")

def extract_lr(cr):
    return cr.replace("Causer:","").replace("Result:","").split("->")

def normalize_cr(cr):
    pair = tuple(extract_lr(cr))
    return pair

def denormalize_cr(crel):
    l,r = crel
    return "Causer:{l}->Result:{r}".format(l=l,r=r)

crel = normalize_cr("Causer:5->Result:50")
print(crel)
cr = denormalize_cr(crel)
print(cr)

('5', '50')
Causer:5->Result:50


In [19]:
class WeightedExamples(object):
    def __init__(self, labels, positive_value):
        self.xs = []
        self.labels = labels
        self.positive_value = positive_value
        self.labels  = defaultdict(list) # list of ints
        self.weights = defaultdict(list) # list of floats
    
    def add(self, x, actual_lbl, weights=None):
        self.xs.append(x)
        for lbl in self.labels:
            val = self.positive_value if lbl == actual_lbl else -1
            self.labels[lbl].append(val)
            weight = 1
            if weights:
                weight = weights[lbl]
            self.weights[lbl].append(weight)
    
    def get_labels_for(self, lbl):
        return self.labels[lbl]
    
    def get_weights_for(self, lbl):
        return self.weights[lbl]

In [20]:
import xgboost
#xgboost.DMatrix?

In [48]:
from sklearn.feature_extraction import DictVectorizer

PARSE_ACTIONS = [
    SHIFT,
    REDUCE,
    LARC,
    RARC,
    SKIP
]

CAUSE_EFFECT = "CAUSE_EFFECT"
EFFECT_CAUSE = "EFFECT_CAUSE"
REJECT       = "REJECT" # Not a CREL

CREL_ACTIONS = [
    CAUSE_EFFECT,
    EFFECT_CAUSE,
    REJECT
]
    
class SearnModel(object):
    def __init__(self, feature_extractor, cr_tags, base_learner_fact, beta_decay_fn = lambda b: b-0.1, positive_val=1):
        # init checks
        #assert CAUSER in tags, "%s must be in tags" % CAUSER
        #assert RESULT in tags, "%s must be in tags" % RESULT
        #assert EXPLICIT in tags, "%s must be in tags" % EXPLICIT

        self.feat_extractor = feature_extractor    # feature extractor (for use later)
        self.positive_val = positive_val
        self.base_learner_fact = base_learner_fact # Sklearn classifier
        
        self.cr_tags = set(cr_tags)                # causal relation tags
        self.epoch = -1
        self.beta = 1.0 # probability of using oracle for each parsing decision, initialize to 1 as we don't use until second epoch
        self.beta_decay_fn = beta_decay_fn
        self.stack = []
        
        self.parser_models = []
        self.current_parser_models = None
        self.current_parser_dict_vectorizer = None
        self.crel_models = []
        self.current_crel_model   = None
        self.current_crel_dict_vectorizer = None
        
        self.training_datasets_parsing = {}
        self.training_datasets_crel = {}
        self.current_model = None
        
    def train(self, tagged_essays, max_epochs):
        
        trained_with_beta0 = False
        for i in range(0, max_epochs):
            if self.beta < 0:
                trained_with_beta0 = True
                
            self.epoch +=1
            print("Epoch: {epoch}".format(epoch=self.epoch))
            print("Beta:  {beta}".format(beta=self.beta))
            
            #TODO - provide option for different model types here?
            parse_examples = WeightedExamples(PARSE_ACTIONS, self.positive_val)
            crel_examples  = WeightedExamples(CREL_ACTIONS,  self.positive_val)
            
            for essay_ix, essay in enumerate(tagged_essays):
                for sent_ix, taggged_sentence in enumerate(essay.sentences):
                    predicted_tags = essay.pred_tagged_sentences[sent_ix]
                    relations = self.parse_sentence(taggged_sentence, predicted_tags, parse_examples, crel_examples)
            
            #TODO, dictionary vectorize examples, train a weighted binary classifier for each separate parsing action
            self.train_parse_models(parse_examples)
            self.train_crel_models(crel_examples)
            
            self.training_datasets_parsing[self.epoch] = parse_examples
            self.training_datasets_crel[self.epoch]    = crel_examples
            
            # Decay beta
            self.beta = self.beta_decay_fn(beta)
            if self.beta < 0 and trained_with_beta0:
                print("beta decayed below 0 - beta:'{beta}', stopping".format(beta=self.beta))
                break
        #end [for each epoch]
        if not trained_with_beta0:
            print("Algorithm hit max epochs without training with beta <= 0 - final_beta:{beta}".format(beta=self.beta))
    
    def train_parse_models(self, examples):
        models = {}
        self.current_parser_dict_vectorizer = DictVectorizer(sparse=True)
        xs = self.current_parser_dict_vectorizer.fit_transform(examples.xs)
        
        for action in PARSE_ACTIONS:
            mdl = self.base_learner_fact()
            ys = examples.get_labels_for(action)
            weights = examples.get_weights_for(action)
            #TODO - train cost sensitive classifier
            mdl.fit(xs,ys)
            models[action] = mdl
        self.parser_models.append(models)
            
    def train_crel_models(self, examples):
        
        self.current_crel_dict_vectorizer = DictVectorizer(sparse=True)
        
        model = self.base_learner_fact()
        xs = self.current_crel_dict_vectorizer.fit_transform(examples.xs)
        ys = examples.get_labels_for(action)
        model.fit(xs,ys)
        
        self.crel_models.append(model)
    
    def add_relation(self, action, tos, buffer, ground_truth, relations):
        if action in [LARC, RARC]:
            if (tos, buffer) in ground_truth:
                relations.add((tos,buffer))
            elif (buffer,tos) in ground_truth:
                relations.add((buffer,tos))
    
    def relations_for_action(self, forced_action, ground_truth, remaining_buffer, oracle):
        oracle = oracle.clone()
        relns = set()
        first_action = True
        for buffer in remaining_buffer:
            while True:
                tos = oracle.tos()
                if not first_action: # need to force first action
                    action = oracle.consult(tos, buffer)
                else:
                    action = forced_action
                    first_action = False
                self.add_relation(action, tos, buffer, ground_truth, relns)                        
                if not oracle.execute(action, tos, buffer):
                    break
                if oracle.is_stack_empty():
                    break
        return relns
        
    def compute_cost(self, ground_truth, remaining_buffer, oracle):
        
        tos = oracle.tos()
        gold_action = oracle.consult(tos, remaining_buffer[0])
        gold_parse = self.relations_for_action(gold_action, ground_truth, remaining_buffer, oracle)
        
        action_costs = {}
        for action in PARSE_ACTIONS:
            if action == gold_action:
                continue
            parse = self.relations_for_action(action, ground_truth, remaining_buffer, oracle)
            num_matches = len(gold_parse.intersection(parse))
            # recall
            false_negatives = len(gold_parse) - num_matches
            # precision
            false_positives = len(parse) - num_matches      
            # Cost is the total of the false positives + false negatives
            cost = false_positives + false_negatives
            action_costs[action] = cost
        
        # Cost of the gold action is the mean of all of the wrong choices
        action_costs[gold_action] = np.mean(list(action_costs.values()))
        return action_costs
        
    def predict_parse_action(self, feats):
        xs = self.current_parser_dict_vectorizer.transform(feats)
        prob_by_label = {}
        for action in PARSE_ACTIONS:
            prob_by_label[action] = self.current_parser_models[action].predict_proba(xs)[0][-1]
        
        max_act, max_prob = max(prob_by_label.items(), key = lambda tpl: tpl[1])
        return max_act
    
    def predict_crel_action(self, feats):
        xs = self.current_crel_dict_vectorizer.transform(feats)
        return self.current_crel_model.predict(xs)
    
    def parse_sentence(self, taggged_sentence, predicted_tags, parse_examples, crel_examples):
   
        action_history = []
        action_tag_pair_history = []

        all_tags = set()
        all_predicted_tags = set()
        
        min_ixs, max_ixs = defaultdict(lambda : len(taggged_sentence)+1 ), defaultdict(lambda : -1)
        ptag_seq = []
        words = []
        for i, (wd, tags) in enumerate(taggged_sentence):
            words.append(wd)
            all_tags.update(tags)
            ptag = predicted_tags[i]
            if ptag == EMPTY_TAG:
                continue
            if not ptag in all_predicted_tags:
                ptag_seq.append(ptag)
            all_predicted_tags.add(ptag)
            # determine span of each predicted tag
            min_ixs[ptag] = min(min_ixs[ptag], i)
            max_ixs[ptag] = max(max_ixs[ptag], i)
            
        ground_truth = all_tags.intersection(self.cr_tags)
        ground_truth = set([normalize_cr(crel) for crel in ground_truth])
        # Filter to only those crels that have support in the predicted tags
        supported_crels = set()
        for crel in ground_truth:
            l,r = crel
            if l in all_predicted_tags and r in all_predicted_tags:
                supported_crels.add(crel)
        ground_truth = supported_crels
        
        # Initialize stack, basic parser and oracle
        stack = Stack(False)        
        stack.push(ROOT)
        parser = Parser(stack)
        oracle = Oracle(ground_truth, parser)
        
        predicted_relations = []
        
        # Oracle parsing logic
        for tag_ix, buffer in enumerate(ptag_seq):
            buffer = buffer
            word_seq = words[min_ixs[buffer]:max_ixs[buffer]+1]
            buffer_feats = self.feat_extractor.extract(buffer, word_seq, self.positive_val)
            while True:
                tos = oracle.tos()
                tos_word_seq = words[min_ixs[tos]:max_ixs[tos]+1]
                tos_feats = self.feat_extractor.extract(tos, tos_word_seq, self.positive_val)
                
                feats = self.get_conditional_feats(action_history, action_tag_pair_history, tos, buffer, ptag_seq[:tag_ix], ptag_seq[tag_ix+1:])
                interaction_feats = self.get_interaction_feats(tos_feats, buffer_feats)
                feats.update(buffer_feats)
                feats.update(tos_feats)
                feats.update(interaction_feats)
                
                gold_action = oracle.consult(tos, buffer)
                
                # Consult Oracle or Model based on coin toss
                rand_float = np.random.random_sample() # between [0,1) (half-open interval, includes 0 but not 1)
                # If no trained models, always use Oracle
                if rand_float >= self.beta and len(self.self.parser_models) > 0:
                    action = self.predict_parse_action(feats)
                else:
                    action = gold_action
                
                action_history.append(action)
                action_tag_pair_history.append((action, tos, buffer))
                
                #TODO compute weight for each example
                #TODO - if errors are made, are there more optimal decisions?
                #TODO - if an error was made on this action, how many crels would be affected?
                    # Can we clone the current oracle and parser state, and determine the max errors if the golden action is not taken
                    # As some wrong decisions can have higher losses (e.g. a Reduce could break 2 relations where a shift may break one or be-recoverable)
                    # Then we are better training multiple binary classifiers so each decision can be weighed independently
                
                cost_per_action = self.compute_cost(ground_truth, ptag_seq[tag_ix:], oracle)
                # make a copy as changing later
                parse_examples.add(dict(feats), gold_action, cost_per_action)

                # Decide the direction of the causal relation
                if action in [LARC, RARC]:
                    if (tos, buffer) in ground_truth:
                        gold_lr_action = CAUSE_EFFECT
                    elif (buffer,tos) in ground_truth:
                        gold_lr_action = EFFECT_CAUSE
                    else:
                        gold_lr_action = REJECT
                    
                    # Add arc to features
                    feats["ARC:" + action] = self.positive_val                    
                    rand_float = np.random.random_sample()
                    if rand_float >= self.beta and len(self.crel_models) > 0:
                        #TODO - we need separate models for different decisions
                        lr_action = self.predict_crel_action(feats)
                    else:
                        lr_action = gold_lr_action
                    
                    if lr_action == CAUSE_EFFECT:
                        predicted_relations.append((tos,buffer))
                    elif lr_action == EFFECT_CAUSE:
                        predicted_relations.append((buffer,tos))
                                            
                    # cost is always 1 for this action (cost of 1 for getting it wrong)
                    #  because getting the wrong direction won't screw up the parse as it doesn't modify the stack

                    crel_examples.append(dict(feats), gold_lr_action)                    
                    # Not sure we want to condition on the actions of this crel model
                    #action_history.append(lr_action)
                    #action_tag_pair_history.append((lr_action, tos, buffer))
                    
                #end if action in [LARC,RARC]
                if not oracle.execute(action, tos, buffer):
                    break
                if oracle.is_stack_empty():
                    break
                    
        pred_relns = [denormalize_cr(cr) for cr in predicted_relations]
        return pred_relns
    
    def get_interaction_feats(self, fts1, fts2):
        interactions = {}
        for fta, vala in fts1.items():
            for ftb, valb in fts2.items():
                if vala > 0 and valb > 0:
                    interactions["inter: " + fta + "|" + ftb] = self.positive_val
        return interactions
            
    def get_conditional_feats(self, action_history, action_tag_pair_history, tos, buffer, previous_tags, subsequent_tags):
        feats = {}
        feats["tos:"        + tos]    = self.positive_val
        feats["buffer:"     + buffer] = self.positive_val
        feats["tos_buffer:" + tos + "|" + buffer] = self.positive_val
        
        ### PREVIOUS TAGS
        for i, tag in enumerate(previous_tags[::-1]):
            feats["prev_tag-{i}:{tag}".format(i=i, tag=tag)] = self.positive_val
            feats["prev_tag:{tag}".format(tag=tag)] = self.positive_val
            
        if len(previous_tags) > 0:
            feats["prev-tag-tos-buffer:{tag}_{tos}_{buffer}".format(tag=previous_tags[-1], tos=tos, buffer=buffer)] = self.positive_val
            feats["prev-tag-buffer:{tag}_{buffer}".format(tag=previous_tags[-1], buffer=buffer)] = self.positive_val
            feats["prev-tag-tos:{tag}_{tos}".format(tag=previous_tags[-1], tos=tos)] = self.positive_val
            bigrams = compute_ngrams(previous_tags, 2, 2)
            for i, bigram in enumerate(bigrams[::-1]):
                feats["prev_bigram-tag-{i}:{tag}".format(i=i, tag=str(bigram))] = self.positive_val
                feats["prev_bigram-tag:{tag}".format(tag=str(bigram))] = self.positive_val

        ### REMAINING TAGS
        for i, tag in enumerate(subsequent_tags):
            feats["subseq_tag-{i}:{tag}".format(i=i, tag=tag)] = self.positive_val
            feats["subseq_tag:{tag}".format(i=i, tag=tag)] = self.positive_val

        if len(subsequent_tags) > 0:
            feats["subseq-tag-tos-buffer:{tag}_{buffer}".format(tag=subsequent_tags[0], tos=tos, buffer=buffer)] = self.positive_val
            feats["subseq-tag-buffer:{tag}_{buffer}".format(tag=subsequent_tags[0], buffer=buffer)] = self.positive_val
            feats["subseq-tag-tos:{tag}_{tos}".format(tag=subsequent_tags[0], tos=tos)] = self.positive_val            
            bigrams = compute_ngrams(subsequent_tags, 2, 2)
            for i, bigram in enumerate(bigrams):
                feats["subseq_bigram-tag-{i}:{tag}".format(i=i, tag=str(bigram))] = self.positive_val
                feats["subseq_bigram-tag:{tag}".format(tag=str(bigram))] = self.positive_val

        # features for each previous action
        action_tally = defaultdict(int)
        for i, action in enumerate(action_history[::-1]):
            feats["action-{i}:{action}".format(i=i, action=action)] = self.positive_val
            feats["action:{action}".format(action=action)] = self.positive_val
            action_tally[action] +=1        
        
        # Features for the number of times each action has been performed
        for action, count in action_tally.items():                           
            feats["action-tally:{action}_{count}".format(action=action, count=count)] = self.positive_val
                                       
        if len(action_history) > 0:
            feats["prev_action-tos-buffer:{action}_{tos}_{buffer}".format(action=action_history[-1], tos=tos, buffer=buffer)] = self.positive_val
            feats["prev_action-buffer:{action}_{buffer}".format(action=action_history[-1], buffer=buffer)] = self.positive_val
            feats["prev_action-tos:{action}_{tos}".format(action=action_history[-1], tos=tos)] = self.positive_val
            bigrams = compute_ngrams(action_history, 2, 2)
            for i, bigram in enumerate(bigrams[::-1]):
                feats["prev_bigram_action-{i}:{tag}".format(i=i, tag=str(bigram))] = self.positive_val
                feats["prev_bigram_action:{tag}".format(tag=str(bigram))] = self.positive_val
        
        for i, (action, prev_tos, prev_buffer) in enumerate(action_tag_pair_history[::-1]):
            feats["actiontag-{i}:{action}_{tos}_{buffer}".format(i=i, action=action, tos= prev_tos, buffer=prev_buffer)] = self.positive_val
            feats["actiontag:{action}_{tos}_{buffer}".format(action=action, tos= prev_tos, buffer=prev_buffer)] = self.positive_val

            feats["actiontos-{i}:{action}_{tos}".format(i=i, action=action, tos=prev_tos)] = self.positive_val
            feats["actiontos:{action}_{tos}".format(action=action, tos=prev_tos)] = self.positive_val
            
            feats["actionbuffer-{i}:{action}_{buffer}".format(i=i, action=action, buffer=prev_buffer)] = self.positive_val
            feats["actionbuffer:{action}_{buffer}".format(action=action, buffer=prev_buffer)] = self.positive_val
        
        if len(action_tag_pair_history) > 0:
            action, prev_tos, prev_buffer = action_tag_pair_history[-1]            
            feats["prev_actiontag_tos_buffer_currnet_tos_current_buffer:{action}_{prev_tos}_{prev_buffer}_{tos}_{buffer}".format(action=action, prev_tos=prev_tos, prev_buffer=prev_buffer, tos=tos, buffer=buffer)] = self.positive_val
            feats["prev_actiontag_tos_buffer_current_buffer:{action}_{prev_tos}_{prev_buffer}_{buffer}".format(action=action, prev_tos=prev_tos, prev_buffer=prev_buffer, buffer=buffer)] = self.positive_val
            feats["prev_actiontag_tos_buffer_current_tos:{action}_{prev_tos}_{prev_buffer}_{tos}".format(action=action, prev_tos=prev_tos, prev_buffer=prev_buffer, tos=tos)] = self.positive_val
            
        return feats

In [49]:
from sklearn.linear_model import LogisticRegression

#class SearnModel(object):
#    def __init__(self, feat_extractor, cr_tags, base_learner_fact, beta, beta_decay_fn = lambda b: b-0.1, positive_val=1):
    
parse_model = SearnModel(feat_extractor, cr_tags, base_learner_fact=LogisticRegression)
parse_model.train(pred_tagged_essays, 2)

Epoch: 0
Beta:  1.0


KeyError: 'root'

## Cost Sensitive Learning
* Cost Proportionate Example weighting
  * https://www.researchgate.net/publication/4047552_Cost-Sensitive_Learning_by_Cost-Proportionate_Example_Weighting
* You Can Set Per Observation Weights in XGBoost
  * http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
  * see weight parameter in dtrain

## TODO
* Need to make sure the tagger tags EXCPLICT tags. These can then be skipped by the parser, but will be included in the features used to train the parser and taggger. Do we want to train a separate tagger that determines if a tagged word is a cause, explict or result. That will then resolve the direction of the relation?